# Lab 2: Linear Models
This lab, like the others that follow it, will be a small, largely self-guided project in building a machine learning model.
You will write a logistic regression classifier, from scratch, to classify 28x28 pixel images of handwritten digits (0 - 9) by which digit appears in the image.
This is [the famous MNIST dataset](http://yann.lecun.com/exdb/mnist/), which has 60,000 training examples and 10,000 test examples.

In [ ]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
# Any other imports you need go here

## Section 0: download the data
Download the data as a CSV [here](https://pjreddie.com/projects/mnist-in-csv/) (the original data format is very unfriendly) and read the data format on the website.

## Section 1: load, understand, and preprocess the data
Load the data from disk into four numpy arrays: x_train (feature vectors from the training set), y_train (labels from the training set), x_test (feature vectors from the test set), and y_test (labels from the test set).

Then, do some simple preprocessing.
Normalize your features by subtracting the mean and dividing by the range.
This is less important for linear models, but is generally good practice and will be much more important with more complex models.
Make sure you don't use any values computed from the test set, just the mean and range of the training set.

Finally, display one of the training images, and print its correct label.

Functions to look at:
 - `np.loadtxt`
 - `np.reshape`
 - `plt.imshow`

In [ ]:
# Your code here

## Section 2: build a data pipeline
Make a pipeline that turns the dataset you have in numpy arrays into tensors that your model can use.
The pipeline should also shuffle and batch the data (with some reasonable batch size; I used 64 because I'm training on a GPU but you might want to go lower).

There are multiple ways to approach this.
No matter what, you'll need two `tf.data.Dataset`s, one for train and one for test, and probably you'll do some transforms to that.
I made datasets which did not use `repeat`, and used reinitializable iterators to feed the data -- you might instead repeat the data for as many epochs (full run-throughs of the dataset during training, ~5-10 for this assignment) as you plan to train for, then use one-shot iterators instead, or do something else.
Don't use feeding to pass in your data, though, since it'll result in a slow pipeline.

Optionally, you might want to cache or prefetch data to prevent it from being loaded multiple times while the model is training and keep the model from needing to wait for data.

Now might also be a good place to convert the labels to one-hot encoding (see below), though you don't need to.
If you want, you could also do your data normalization on-the-fly here (but make sure it still does the same thing).

Functions to look at (not exhaustive):
 - `tf.data.Dataset.from_tensor_slices`
 - `tf.data.Dataset.shuffle`
 - `tf.data.Dataset.batch`
 - `tf.data.Dataset.repeat`
 - `tf.data.Dataset.make_one_shot_iterator`
 - `tf.data.Dataset.make_initializable_iterator`
 - `tf.data.Iterator.from_structure`
 - `tf.data.Iterator.make_initializer`

In [ ]:
# Your code here

## Section 3: build a model graph
This is where the actual model gets built.
I'll give you the steps, but there's flexibility in how you implement each part.
Make sure your design is clean, both for efficiency and ease of debugging.
Names and name scopes help divide the model into logical parts.

Note that the first axis of every tensor will be the batch size.
So, a tensor that in your model is a scalar will likely have a shape of (?), and a 10-element vector will have a shape of (?, 10).
The "?" indicates that TensorFlow will treat this axis as being variable-length, since it can't infer it just from context.
It may be helpful to switch your thinking back and forth from one view (no batches, scalars are scalars) when thinking about modeling to another (scalars come in batches) when writing code that changes shapes.

If you're having difficulty debugging, try looking at your graph in TensorBoard, printing tensor objects to see their shapes, and running small parts of the graph in a session while feeding values to certain tensors.

Functions to look at throughout:
 - `tf.cast`
 - `tf.expand_dims`
 - `tf.squeeze`
 - arithmetic operations

### 3.1: Input tensors
Get a tensor for the input image (features) and another tensor for the correct label.
It may be convenient to convert the label to a one-hot vector: 10 elements, each of which is 0 except the place of the correct label.
For instance, a label of "3" would be the vector $$[0, 0, 0, 1, 0, 0, 0, 0, 0, 0].$$

Functions to look at:
 - `tf.data.Iterator.get_next`
 - `tf.one_hot`

In [ ]:
# Your code here

### 3.2: Variables
Logistic regression needs two variables, correctly shaped: a weight matrix and a vector of biases.

In [ ]:
# Your code here

### 3.3: Logits
Compute the inputs to the softmax functiton, the logits or "unnormalized probabilities."
The result should be, for each example in the batch, a vector of ten values (so the tensor has a shape like (?, 10)).

In [ ]:
# Your code here

### 3.4: Softmax probabilities
Apply the softmax function to the logits to obtain a vector of class probabilities.
Again, there should be a probability for each class for each example.
Do _not_ use helper functions to compute the softmax function, like `tf.nn.softmax` or `tf.contrib.layers.softmax`.
Instead, stick to core operations like `tf.exp`.

Two main reasons for this:
 1. Better to learn without the API doing too much of the work for you; when doing model development it'll basically all be from scratch
 2. I couldn't get `tf.nn.softmax` to run on a GPU, so I'd do it this way in practice too

In [ ]:
# Your code here

### 3.5: Cross-entropy loss
Compute the per-example cross-entropy loss $$L = -y \cdot \log p_\text{model}(y)$$ using the probabilities and the correct label.

Then, take the mean of the per-example losses to compute a per-batch loss.
Also, add a summary operation which saves the per-batch loss so you can plot it later.

In [ ]:
# Your code here

### 3.6: The optimizer
Add an optimizer (simple gradient descent is fine) and an operation to minimize the per-batch loss.
You may need to play around with the learning rate to find one that works.

In [ ]:
# Your code here

### 3.7: Predicted label and per-batch accuracy
The model should predict the digit it assigns the highest probability. 
Add a tensor which represents the predicted digit; this lets you use the model "in the wild".
Then, add a tensor which represents what fraction of the batch the model predicted correctly (its accuracy, or average 0/1 loss), and a summary operation for accuracy.

Finally, add a tensor to merge the summaries made so far.
Or, just add them one at a time if you want (but why would you?)

Functions to look at:
 - `tf.argmax`
 - `tf.equal`
 - `tf.reduce_mean`
 - `tf.summary.merge_all`

In [ ]:
# Your code here

## Section 4: train the model
Make a training loop which iterates through the full training set multiple times, and, for each batch:
 1. Runs the training operation
 2. Adds the summaries to a training FileWriter, with the correct batch number

After each epoch, iterate over the test dataset, adding summaries to a test FileWriter, and print the average test accuracy over the whole test set.
Finally, use a `Saver` to save the whole graph to disk.

Remember to: 
 - Initialize variables
 - Give FileWriters references to the graph
 
Then, run training.
You should expect the model to hit about 90% test-set accuracy.
Not bad for a linear model!
If it doesn't, it might indicate a bug in your code, or you might need to tune hyperparameters (e.g. batch size, learning rate).
The accuracy should sharply jump after the first epoch, so you don't need to wait for the full model to train to know you have a bug. 

You might notice, looking at the training plots on TensorBoard, how close the training and test errors are.
This suggests our model is probably underfitting, and a more powerful model will do better.

<sup><sub>My model was showing 20% accuracy and it took over an hour to figure out I was actually just computing accuracy wrong :(</sub></sup>

In [ ]:
# Your code here

## Section 5: use the model for inference
Pull one or more examples from the test set and display its image.
Then, use your model to predict what digit the image is, print that, and print the true label (which hopefully matches).

Remember you'll need to use `Saver.restore()` to load the trained model before inference.

In [ ]:
# Your code here

## Section 6: visualizations
Visualize, as an image (i.e. `plt.imshow`), the weight matrix for each of the digits.
You might want to use the `vmin` and `vmax` arguments to put all of the matrices on the same scale. 
You'll need to load the weights again to save them to numpy arrays (by `run()`-ing the variables).

The results are pretty interesting.
The image should be brightest where the weights strongly indicate that digit, so you can see what's (linearly) characteristic of different digits -- I see a strong dark spot in the middle for zero, a distinctive curly tail on 2, and a consistent 3.
Other digits (e.g. 8 and 9) have much weaker patterns, suggesting greater variation in how people draw them.

In [ ]:
# Your code here

## Section 7: Keras
Finally, make the same model with Keras and train it.
You should again see about 90% test-set accuracy.

Hint: logistic regression is just linear regression followed by the softmax function.

Functions to look at:
 - `keras.utils.to_categorical` (for making one-hot labels)
 - `keras.layers.Dense`
 - `keras.layers.Activation`

In [ ]:
# Your code here